# Entrenamiento de RNNs en una Serie de Tiempo

En esta sección emplearemos redes neuronales recurrentes para modelar series de tiempo, es decir una serie
de registros (tı́picamente valores reales) regularmente indexados en el tiempo. Para ello utilizaremos un dataset
denominado "*Minimum Daily Temperatures*", el cual describe la temperatura mínima diaria en un período de 10 años (1981 a 1990) en la ciudad de Melbourne, Australia. Las unidades de las 3670 observaciones fueron medidas en grados celsius. A continuación se muestra la secuencia de tiempo:
<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2016/11/Minimum-Daily-Temperatures.png" width="60%" />



La tarea predictiva consiste en estimar la temperatura mínima diaria de algún día basado en la información de días anteriores.  
*La fuente es acreditada a Australian Bureau of Meteorology.*

In [11]:
import pandas as pd
import numpy as np

> a) Escriba una función que cargue los datos, los divida en 1500 de entrenamiento y el resto (500) de pruebas. Además de esto escálelos apropiadamente para trabajar con redes recurrentes.

Ojo no se usan todos los datos solo 200

In [3]:
#cargar los datos y dividir
name_f = "time_series_data.csv"
dataframe = pd.read_csv(name_f,sep=',',usecols=[1],engine='python',skipfooter = 3)[:2000]
dataframe[:] = dataframe[:].astype('float32')
df_train, df_test = dataframe[:1500].values, dataframe[1500:].values

#escalar datos
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1)).fit(df_train)
stream_train_scaled = scaler.transform(df_train)
stream_test_scaled = scaler.transform(df_test)

In [21]:
print(dataframe.shape)
dataframe.describe()

(2000, 1)


,"Daily minimum temperatures in Melbourne, Australia, 1981-1990"
count,2000.000000
mean,11.148250
std,4.194983
min,0.000000
25%,8.100000
50%,11.000000
75%,14.000000
max,26.299999


In [23]:
n_train = df_train.shape[0]
n_test = df_test.shape[0]
print("Datos de entrenamiento: ", n_train, ", correspondiente al ", n_train/dataframe.shape[0], "% de los datos")
print("Datos de entrenamiento: ", n_test, ", correspondiente al ", n_test/dataframe.shape[0], "% de los datos")

Datos de entrenamiento:  1500 , correspondiente al  0.75 % de los datos
Datos de entrenamiento:  500 , correspondiente al  0.25 % de los datos


> b) Ahora nos gustarı́a manipular los datos, para que hagamos la predicción de la temperatura mínima para el tiempo siguiente usando la temperatura mínima de los últimos perı́odos de tiempo. El número de perı́odos de tiempos que usaremos se denomina lag. Por ejemplo, tendremos un lag igual a 3, si para predecir el valor $x_{t+1}$ en el tiempo siguiente usamos la información del tiempo actual $x_t$ y la de los dos perı́odos anteriores $x_{t-1}$ y $x_{t-2}$ como variables de entrada. Realice una función que reciba una secuencia de valores y la transforme en dos arreglos dataX (inputs) y dataY (targets) donde el número de caracterı́sticas de la la matriz de entrada (columnas) sea el número de tiempos que se considerarán como información (lag).

In [15]:
#modificar
def create_dataset(dataset,lag=1):
    #preprocesamiento asumiendo dataset es una secuencia
    dataX = []
    dataY =[]
    for t_plus_1 in range(lag,len(dataset)):
        #se crea el Y (target)
        predecir = dataset[t_plus_1]
        dataY+= list(predecir) 
        
        #se crea el X (inputs) columnas para predecir Y
        columnasX = []
        for i in np.arange(lag,0,-1): #para los valores anteriores al t_plus_1 durante un lag
            columnasX += list( dataset[t_plus_1-i] )
        dataX.append(columnasX)
    return np.array(dataX),np.array(dataY)

data_x, data_y = create_dataset(stream_train_scaled, 3)

In [17]:
data_x[:10]

array([[0.7870723 , 0.6806084 , 0.7148289 ],
       [0.6806084 , 0.7148289 , 0.5551331 ],
       [0.7148289 , 0.5551331 , 0.6007605 ],
       [0.5551331 , 0.6007605 , 0.6007605 ],
       [0.6007605 , 0.6007605 , 0.6007605 ],
       [0.6007605 , 0.6007605 , 0.661597  ],
       [0.6007605 , 0.661597  , 0.82889736],
       [0.661597  , 0.82889736, 0.7604563 ],
       [0.82889736, 0.7604563 , 0.61596966],
       [0.7604563 , 0.61596966, 0.50570345]], dtype=float32)